In [19]:
import pandas as pd
from cleaning_function import *
data = pd.read_csv('cvrm_master.csv')

/var/folders/26/l89mqgp17zj82yvlmd3z3r200000ks/T/ipykernel_6949/187753911.py:3: DtypeWarning: Columns (1697,1698) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('cvrm_master.csv')


In [20]:
data = drop_null_columns(data, 0.6)
data.replace([np.inf, -np.inf], np.nan, inplace=True)

In [21]:
threshold = 0.05 * data.shape[1] 
data_df_filtered = data[data.isna().sum(axis=1) < threshold]
data_df_filtered

,statecounty_x,"AirPollutant: 1,3-butadiene_ug/m^3_EPA2018",AirPollutant: Acetaldehyde_ug/m^3_EPA2017,AirPollutant: Acetaldehyde_ug/m^3_EPA2018,AirPollutant: Benzene_ug/m^3_EPA2017,AirPollutant: Benzene_ug/m^3_EPA2018,AirPollutant: Carbon tetrachloride_ug/m^3_EPA2017,AirPollutant: Carbon tetrachloride_ug/m^3_EPA2018,AirPollutant: Formaldehyde_ug/m^3_EPA2017,AirPollutant: Formaldehyde_ug/m^3_EPA2018,...,County Seat(s) [3],Population (2010),Land Area km²,Land Area mi²,Water Area km²,Water Area mi²,Total Area km²,Total Area mi²,Latitude,Longitude
0,01autauga,0.01,1.50,1.41,0.19,0.18,0.51,0.37,1.84,1.69,...,Prattville,54571,1539.582,594.436,25.776,9.952,1565.358,604.388,32.536382,-86.644490
1,01baldwin,0.01,1.15,1.10,0.17,0.16,0.51,0.37,1.47,1.40,...,Bay Minette,182265,4117.522,1589.784,1133.19,437.527,5250.712,2027.311,30.659218,-87.746067
2,01barbour,0.02,1.51,1.39,0.18,0.17,0.50,0.37,1.83,1.64,...,Clayton,27457,2291.819,884.876,50.865,19.639,2342.684,904.515,31.870670,-85.405456
4,01blount,0.01,1.24,1.19,0.20,0.19,0.50,0.37,1.65,1.59,...,Oneonta,57322,1669.962,644.776,15.157,5.852,1685.119,650.628,33.977448,-86.567246
6,01butler,0.01,1.51,1.41,0.15,0.15,0.50,0.37,1.77,1.63,...,Greenville,20947,2011.977,776.829,2.727,1.053,2014.704,777.882,31.751667,-86.681969
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3119,55wood,0.01,0.64,0.56,0.19,0.25,0.52,0.38,0.73,0.73,...,Wisconsin Rapids,74749,2054.162,793.116,42.281,16.325,2096.443,809.441,44.461413,-90.038825
3126,56fremont,0.00,0.34,0.28,0.07,0.08,0.42,0.31,0.48,0.46,...,Lander,40123,23785.968,9183.814,212.131,81.904,23998.099,9265.718,43.055303,-108.605531
3130,56laramie,0.00,0.33,0.29,0.09,0.10,0.43,0.32,0.54,0.51,...,Cheyenne,91738,6956.480,2685.912,4.27,1.649,6960.749,2687.561,41.292830,-104.660395
3132,56natrona,0.00,0.37,0.31,0.09,0.10,0.43,0.32,0.60,0.56,...,Casper,75450,13831.447,5340.352,91.461,35.313,13922.908,5375.665,42.973641,-106.764877


In [22]:
data_geo_knn = pd.read_csv('geoknn.csv')
data_knn = pd.read_csv('knnnormal.csv')
data_median = pd.read_csv('median_imputation.csv')
data_mean = pd.read_csv('mean_imputation.csv')
data_graphsage = pd.read_csv('graphsage_df.csv')

/var/folders/26/l89mqgp17zj82yvlmd3z3r200000ks/T/ipykernel_6949/458870595.py:1: DtypeWarning: Columns (1517,1518) have mixed types. Specify dtype option on import or set low_memory=False.
  data_geo_knn = pd.read_csv('geoknn.csv')
/var/folders/26/l89mqgp17zj82yvlmd3z3r200000ks/T/ipykernel_6949/458870595.py:2: DtypeWarning: Columns (1513,1514) have mixed types. Specify dtype option on import or set low_memory=False.
  data_knn = pd.read_csv('knnnormal.csv')
/var/folders/26/l89mqgp17zj82yvlmd3z3r200000ks/T/ipykernel_6949/458870595.py:3: DtypeWarning: Columns (1513,1514) have mixed types. Specify dtype option on import or set low_memory=False.
  data_median = pd.read_csv('median_imputation.csv')
/var/folders/26/l89mqgp17zj82yvlmd3z3r200000ks/T/ipykernel_6949/458870595.py:4: DtypeWarning: Columns (1513,1514) have mixed types. Specify dtype option on import or set low_memory=False.
  data_mean = pd.read_csv('mean_imputation.csv')


In [23]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error

# Load original dataset
data_original = data

# Load imputed datasets
data_geo_knn = pd.read_csv("geoknn.csv")
data_knn = pd.read_csv("knnnormal.csv")
data_median = pd.read_csv("median_imputation.csv")
data_mean = pd.read_csv("mean_imputation.csv")
data_graphsage = pd.read_csv("graphsage_df.csv")

# **Step 1: Find Common Numeric Columns**
numeric_cols = data_original.select_dtypes(include=[np.number]).columns

# Find the common columns in all datasets
common_cols = set(numeric_cols)
for dataset in [data_geo_knn, data_knn, data_median, data_mean, data_graphsage]:
    common_cols &= set(dataset.columns)  # Keep only columns present in all datasets

common_cols = list(common_cols)  # Convert back to a list

# **Step 2: Keep Only These Columns**
data_original = data_original[common_cols]
data_geo_knn = data_geo_knn[common_cols]
data_knn = data_knn[common_cols]
data_median = data_median[common_cols]
data_mean = data_mean[common_cols]
data_graphsage = data_graphsage[common_cols]

# **Step 3: Create a Test Set with Artificially Removed Values**
np.random.seed(42)  # Ensure reproducibility
test_fraction = 0.05  # Remove 5% of known values

# Select a random 5% of known (non-NaN) values and mask them
masked_data = data_original.copy()
mask = (np.random.rand(*masked_data.shape) < test_fraction) & masked_data.notna()
masked_data[mask] = np.nan  # Introduce artificial missing values

# **Step 4: Compute MSE for Imputation Methods**
def compute_mse(original, imputed, mask):
    mse_scores = {}
    for col in original.columns:
        if col in imputed.columns:
            # Get only the artificially removed values
            valid_indices = mask[col]
            if valid_indices.any():
                y_true = original.loc[valid_indices, col]
                y_pred = imputed.loc[valid_indices, col]
                
                # Ensure y_true and y_pred are numeric and have matching shapes
                if len(y_true) > 0 and len(y_true) == len(y_pred):
                    mse_scores[col] = mean_squared_error(y_true, y_pred)
    
    return mse_scores

# Compute MSE for each method
mse_geo_knn = compute_mse(data_original, data_geo_knn, mask)
mse_knn = compute_mse(data_original, data_knn, mask)
mse_median = compute_mse(data_original, data_median, mask)
mse_mean = compute_mse(data_original, data_mean, mask)
mse_graphsage = compute_mse(data_original, data_graphsage, mask)

# Combine results into a DataFrame
mse_results = pd.DataFrame({
    "GeoKNN": mse_geo_knn,
    "KNN": mse_knn,
    "Median": mse_median,
    "Mean": mse_mean,
    "GraphSAGE": mse_graphsage
})

# **Step 5: Summarize & Rank the Methods**
mse_summary = mse_results.mean().sort_values()



/var/folders/26/l89mqgp17zj82yvlmd3z3r200000ks/T/ipykernel_6949/3235804116.py:9: DtypeWarning: Columns (1517,1518) have mixed types. Specify dtype option on import or set low_memory=False.
  data_geo_knn = pd.read_csv("geoknn.csv")
/var/folders/26/l89mqgp17zj82yvlmd3z3r200000ks/T/ipykernel_6949/3235804116.py:10: DtypeWarning: Columns (1513,1514) have mixed types. Specify dtype option on import or set low_memory=False.
  data_knn = pd.read_csv("knnnormal.csv")
/var/folders/26/l89mqgp17zj82yvlmd3z3r200000ks/T/ipykernel_6949/3235804116.py:11: DtypeWarning: Columns (1513,1514) have mixed types. Specify dtype option on import or set low_memory=False.
  data_median = pd.read_csv("median_imputation.csv")
/var/folders/26/l89mqgp17zj82yvlmd3z3r200000ks/T/ipykernel_6949/3235804116.py:12: DtypeWarning: Columns (1513,1514) have mixed types. Specify dtype option on import or set low_memory=False.
  data_mean = pd.read_csv("mean_imputation.csv")


In [24]:
mse_summary

GeoKNN       0.0
KNN          0.0
Median       0.0
Mean         0.0
GraphSAGE    0.0
dtype: float64